In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import scipy
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
import pandas as pd
from skimage.feature import peak_local_max

all_data={}
my_array=np.zeros((250,250,400))
data03=pd.read_csv("D:Desktop/thesis/results/new_dosetomedium__clear_OB_waterphantom200MeV__1.csv",delimiter= ',',comment='#',names=['x','y','z','dose'])
for i in np.arange(2,21):
    data1=pd.read_csv("D:Desktop/thesis/results/new_dosetomedium__clear_OB_waterphantom200MeV__"+str(i)+".csv",delimiter= ',',comment='#',names=['x','y','z','dose'])
    data03.dose=data03.dose + data1.dose
data03.dose=data03.dose/(1.60218e-19)*(6.8*10**-9)/20
for i in np.arange(0,len(data03.dose)):
    my_array[data03.x[i]][data03.y[i]][data03.z[i]]=data03.dose[i]
all_data[j]=np.array(my_array)

OB= all_data[200]
OB=OB.T

In [ ]:
all_data={}
my_array=np.zeros((250,250,400))
data03=pd.read_csv("D:Desktop/thesis/results/new_gammaprimary_clear_OB_waterphantom200MeV__1.csv",delimiter= ',',comment='#',names=['x','y','z','dose'])
for i in np.arange(2,21):
    data1=pd.read_csv("D:Desktop/thesis/results/new_gammaprimary_clear_OB_waterphantom200MeV__"+str(i)+".csv",delimiter= ',',comment='#',names=['x','y','z','dose'])
    data03.dose=data03.dose + data1.dose
data03.dose=data03.dose/(1.60218e-19)*(6.8*10**-9)/20
for i in np.arange(0,len(data03.dose)):
    my_array[data03.x[i]][data03.y[i]][data03.z[i]]=data03.dose[i]
all_data[j]=np.array(my_array)

OB= all_data[200]
OB=OB.T

In [16]:
%matplotlib qt


font = {'family' : 'Arial',
        'weight' : 'normal',
        'size'   : 34}

matplotlib.rc('font', **font)
matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=["#54bebe","#c80064"])



x=np.arange(0,250)*0.1
y=np.arange(0,250)*0.1
z=(np.arange(0,400)*0.25)



def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

def gauss_fit(xdata,ydata):
    mean = sum(xdata * ydata) / sum(ydata)
    sigma = np.sqrt(sum(ydata * (xdata - mean)**2) / sum(ydata))
    popt,pcov = curve_fit(gaus,xdata,ydata,p0=[max(ydata), mean, sigma])
    #plt.plot(np.arange(0,len(ydata))+(124-100),gaus(xdata,*popt),label='Gaussian fit',lw=3)
    return popt[-1],np.sqrt(np.diag(pcov))[-1]
    
sigmax=[]
sigmay=[]
for i in np.arange(0,400):
    sigmax.append(gauss_fit(x[124-100:124+100],OB[i,124,124-100:124+100]))
    sigmay.append(gauss_fit(y[124-100:124+100],OB[i,124-100:124+100,124]))
sigmax=np.array(sigmax)
sigmay=np.array(sigmay)
    
filmlocs=np.arange(13,63+1,5)
figure(1,figsize=(8,8))
plt.plot(z,sigmax[:,0],label='X sigma',lw=5)
plt.plot(z,sigmay[:,0],label='Y sigma',lw=5)
plt.fill_between(z,sigmax[:,0]-sigmax[:,1],sigmax[:,0]+sigmax[:,1],alpha=0.4)
plt.fill_between(z,sigmay[:,0]-sigmay[:,1],sigmay[:,0]+sigmay[:,1],alpha=0.4)
plt.grid(c='lightgray')
plt.ylabel('Sigma (mm)')
plt.xlabel('Depth in water (mm)')

filmsigmax=np.load('D:Desktop/thesis/filmsigmax.npy')
filmsigmay=np.load('D:Desktop/thesis/filmsigmay.npy')
plt.errorbar(filmlocs,filmsigmax[:,0],yerr=filmsigmax[:,1],fmt='o',label='Film x sigma',c='#3b7a7a',markersize=12,capsize=8,capthick=2)
plt.errorbar(filmlocs,filmsigmay[:,0],yerr=filmsigmay[:,1],fmt='o', label='Film y sigma',c="#821743",markersize=12,capsize=8,capthick=2)
#l1 = plt.legend(bbox_to_anchor=(1.04, 1), borderaxespad=0)
#l2 = plt.legend(bbox_to_anchor=(1.04, 0), loc="lower left", borderaxespad=0)
#l3 = plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0)
#l4 = plt.legend(bbox_to_anchor=(0, 1.02, 1, 0.2), loc="lower left", mode="expand", borderaxespad=0, ncol=2)

# rx=np.array([4.285,4.393,4.470,4.582,4.6,4.754,4.821,4.926,5.00,5.07])
# ry=np.array([4.544,4.614,4.674,4.782,4.807,4.93,4.986,5.081,5.154,5.225])
# plt.plot(np.arange(140,140+10*10,10),rx,label='Film sigma x',marker='o')
# plt.plot(np.arange(140,140+10*10,10),ry,label='Film sigma y',marker='o')
# plt.legend()
plt.xlim(0,100)
plt.ylim(3.5,5.5)



# print(scipy.stats.linregress(filmlocs,filmsigmax))
# print(scipy.stats.linregress(filmlocs,filmsigmay))
# print(scipy.stats.linregress(z[52:252],sigmax[52:252]))
# print(scipy.stats.linregress(z[52:252],sigmay[52:252]))

np.save('D:Desktop/thesis/sims_gauss_y1.npy',np.mean(OB[52,124-100:124+100,123:126],axis=1))
np.save('D:Desktop/thesis/sims_gauss_y2.npy',np.mean(OB[252,124-100:124+100,123:126],axis=1))
np.save('D:Desktop/thesis/sims_gauss_x1.npy',np.mean(OB[52,123:126,124-100:124+100],axis=0))
np.save('D:Desktop/thesis/sims_gauss_x2.npy',np.mean(OB[252,123:126,124-100:124+100],axis=0))

np.save('D:Desktop/thesis/sims_gauss_x3.npy',np.mean(OB[132,123:126,124-100:124+100],axis=0))
np.save('D:Desktop/thesis/sims_gauss_y3.npy',np.mean(OB[132,124-100:124+100,123:126],axis=1))


OBcax=[]
OBcax_err=[]
for k in np.arange(0,len(z)):
    OBcax.append(np.mean([OB[k,i,j] for i in np.arange(124-8,124+8) for j in np.arange(124-8,124+8)]))
    OBcax_err.append(np.std([OB[k,i,j] for i in np.arange(124-8,124+8) for j in np.arange(124-8,124+8)])/np.sqrt(len([OB[k,i,j] for i in np.arange(124-8,124+8) for j in np.arange(124-8,124+8)])))
OBcax=np.array(OBcax)
OBcax_err=np.array(OBcax_err)


obfilm=np.load('D:Desktop/thesis/200MeVOB.npy',allow_pickle=True)
figure(3,figsize=(8,8))
plt.plot(z,OBcax,lw=5,label='Simulation')
plt.fill_between(z,OBcax-OBcax_err,OBcax+OBcax_err,alpha=0.5)
plt.errorbar(obfilm[1],obfilm[2],yerr=obfilm[3],fmt='o',label='Films',markersize=12,capsize=7,capthick=2)
plt.grid(c='lightgray')
plt.xlabel('Depth in water (mm)')
plt.ylabel('CAX dose (Gy)')
plt.legend()
plt.xlim(0,100)
plt.ylim(8,16)

xsigdiff=[]
ysigdiff=[]
CAXdiff=[]
for i in np.arange(0,11):
    xsigdiff.append(np.abs((filmsigmax[i,0]-sigmax[52+i*20,0])/filmsigmax[i,0]*100))
    ysigdiff.append(np.abs((filmsigmay[i,0]-sigmay[52+i*20,0])/filmsigmay[i,0]*100))
    CAXdiff.append(np.abs((obfilm[2][i]-OBcax[52+i*20])/obfilm[2][i]*100))
    
print(xsigdiff,ysigdiff,CAXdiff)
print(np.mean(xsigdiff),np.mean(ysigdiff),np.mean(CAXdiff))

figure(17)
plt.imshow(OB[52])

[1.4864676961991972, 0.7156877240348606, 0.30497278516260684, 0.008286018824333101, 1.1560733865322232, 0.21126223762506716, 0.13991702659854677, 0.2493182348556782, 0.32472984873464655, 0.029400919374618438, 0.6124276269982193] [0.06350475275125368, 1.493860541223428, 1.4958312769885462, 0.18284976381442486, 0.8356662891740144, 0.2535142979041423, 0.23827577783050938, 0.07773535049328922, 0.2488569356110333, 0.22329237278037262, 0.8114102749410813] [4.772715348069384, 5.773693719309524, 4.115469570441359, 3.935441900073443, 5.5869985872608865, 1.9374203882158048, 1.509529245874195, 1.7033580297703625, 2.4578576260211684, 1.4240755191867378, 3.011729656585196]
0.47623122772181786 0.5386179666829176 3.2934808718916417
